In [2]:
import pandas as pd
from sqlalchemy import create_engine
import pyodbc
import numpy as np
import psycopg2
from datetime import datetime
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from mlxtend.frequent_patterns import fpgrowth

# Define your RDS instance connection parameters
host = "projects-database.cxgcu68ksihx.us-east-1.rds.amazonaws.com"          # RDS endpoint, e.g., <your-db>.rds.amazonaws.com
port = "5432"                       # Default port for PostgreSQL
dbname = "postgres"               # Database name
user = "postgres"                   # Username
password = "DATA602_project"        # Password
try:
    connection = psycopg2.connect(
        host=host,
        port=port,
        database=dbname,
        user=user,
        password=password
    )

    cursor = connection.cursor()

    cursor.execute("SELECT * FROM sales_data;")

    results = cursor.fetchall()

    colnames = [desc[0] for desc in cursor.description]

    # Create a Pandas DataFrame
    ret = pd.DataFrame(results, columns=colnames)

    #for row in results:
        #print(row)

except Exception as error:
    print(f"Error connecting to PostgreSQL: {error}")
finally:
    if connection:
        cursor.close()
        connection.close()
        print("PostgreSQL connection is closed")
        results
ret.head()

def frozenset_to_string(fset):
    if isinstance(fset, frozenset):
        return ', '.join(str(item) for item in fset)  # Convert each item to string
    elif isinstance(fset, (set, list)):
        return ', '.join(str(item) for item in fset)
    else:
        return str(fset)

def frozenset_to_string(fset):
    if isinstance(fset, frozenset):
        return ', '.join(map(str, fset))  # Convert each item in frozenset to string
    return fset

# Helper function to retrieve MRP values for a list of Product IDs
def get_mrp_values(product_ids, ret_df):
    # Check if product_ids is a frozenset, list, or other iterable
    if isinstance(product_ids, frozenset):
        product_ids_list = list(product_ids)  # Convert to list
    else:
        product_ids_list = [int(pid.strip()) for pid in product_ids.split(',')]

    # Fetch MRP values for the provided product IDs
    mrp_values = ret_df[ret_df['description'].isin(product_ids_list)]['unitprice'].tolist()
    return mrp_values


# Function to filter rows where antecedent MRP is less than consequent MRP
def filter_up_sell_candidates(up_sell_df, ret_df):
    valid_rows = []

    for _, row in up_sell_df.iterrows():
        # Get the product IDs for antecedents and consequents
        antecedents = row['antecedents']
        consequents = row['consequents']

        # Fetch MRP values for both antecedents and consequents
        antecedent_mrps = get_mrp_values(antecedents, ret_df)
        consequent_mrps = get_mrp_values(consequents, ret_df)

        # Check if all antecedent MRP values are less than all consequent MRP values
        if max(antecedent_mrps) < min(consequent_mrps):
            valid_rows.append(row)

    # Return the valid rows as a new dataframe
    return pd.DataFrame(valid_rows)

all_cross_sell_results = []
all_up_sell_results = []

PostgreSQL connection is closed


In [3]:
pip install pyodbc

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [4]:
ret

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,index,invoiceno,stockcode,description,quantity,invoicedate,unitprice,customerid,country,category,column1
0,1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,Lantern,1
1,2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,Coat Hanger,2
2,3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,Water Bottle,3
3,5,536365,22752,SET 7 BABUSHKA NESTING BOXES,2,2010-12-01 08:26:00,7.65,17850,United Kingdom,Nesting Box,5
4,7,536366,22633,HAND WARMER UNION JACK,6,2010-12-01 08:28:00,1.85,17850,United Kingdom,Hand Warmer,7
...,...,...,...,...,...,...,...,...,...,...,...
281099,521720,581587,23256,CHILDRENS CUTLERY SPACEBOY,4,2011-12-09 12:50:00,4.15,12680,France,Cutlery,521720
281100,521721,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680,France,Napkins,521721
281101,521723,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680,France,Cutlery,521723
281102,521724,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680,France,Cutlery Set,521724


In [20]:
pd.set_option('display.max_rows', None)  # Show all rows
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.width', None)  # Auto adjust the width to fit the content
pd.set_option('display.max_colwidth', None)  # No truncation of column content
pd.set_option('display.max_seq_item', None)  # No truncation of sequence items (like lists)
pd.set_option('display.float_format', '{:,.2f}'.format)  # Format for floating point numbers, optional
def get_category(item, product_category_map):
    # Fetch category from the product_category_map
    return product_category_map.get(item, None)
# Loop over each unique Store_ID
for country in ret['country'].unique():
    # Filter data for the current Store_ID
    country_data = ret[ret['country'] == country]

    # Prepare the basket as before
    basket = (
        country_data.groupby(['invoiceno', 'description'])['quantity'].sum()
        .unstack()
        .reset_index()
        .fillna(0)
        .set_index('invoiceno')
    )

    # Encode the basket for frequent pattern mining
    my_basket_sets = basket.map(lambda x: 1 if x > 0 else 0)

    # Generate frequent itemsets and association rules
    my_frequent_itemsets = fpgrowth(my_basket_sets, min_support=0.02, use_colnames=True)
    num_itemsets = my_frequent_itemsets.shape[0]  # Number of rows in frequent itemsets dataframe
    my_rules = association_rules(my_frequent_itemsets, metric="lift", min_threshold=0.01,num_itemsets=num_itemsets)

    # Sort cross-sell rules by lift and confidence
    my_cross_sell_rules = my_rules.sort_values(by=['lift', 'confidence'], ascending=[False, False])

    # Convert frozensets to strings
    my_cross_sell_rules['antecedents'] = my_cross_sell_rules['antecedents'].apply(frozenset_to_string)
    my_cross_sell_rules['consequents'] = my_cross_sell_rules['consequents'].apply(frozenset_to_string)

    # Add Store_ID to the cross-sell rules
    my_cross_sell_rules['country'] = country

    # Append cross-sell rules for this store to the list
    all_cross_sell_results.append(my_cross_sell_rules)

    # Merge store data with product categories
    ups = pd.merge(
        country_data[['description', 'unitprice']],
        ret[['description', 'category']],
        on='description'
    )

    # Create a dictionary for fast category lookups
    product_category_map = dict(zip(ups['description'], ups['category']))

    # Add category information to the rules
    my_rules['antecedent_category'] = my_rules['antecedents'].apply(lambda x: get_category(list(x)[0], product_category_map))
    my_rules['consequent_category'] = my_rules['consequents'].apply(lambda x: get_category(list(x)[0], product_category_map))

    # Filter up-sell candidates with same category
    up_sell_candidates = my_rules[my_rules['antecedent_category'] == my_rules['consequent_category']]
    up_sell_candidates_sorted = up_sell_candidates.sort_values(by=['confidence', 'lift'], ascending=[False, False])

    # Filter up-sell candidates where antecedent MRP < consequent MRP
    filtered_up_sell_candidates = filter_up_sell_candidates(up_sell_candidates_sorted, country_data)

    # Add Store_ID to the up-sell candidates
    filtered_up_sell_candidates['country'] = country

    # Append up-sell candidates for this satore to the list
    all_up_sell_results.append(filtered_up_sell_candidates)

# Concatenate all cross-sell and up-sell results into their respective dataframes
final_cross_sell_results = pd.concat(all_cross_sell_results, ignore_index=True)
final_up_sell_results = pd.concat(all_up_sell_results, ignore_index=True)
final_up_sell_results['antecedents'] = final_up_sell_results['antecedents'].apply(frozenset_to_string)
final_up_sell_results['consequents'] = final_up_sell_results['consequents'].apply(frozenset_to_string)

# Display the final results for both cross-sell and up-sell
print("Final Cross-Sell Rules with Country Labels:")
print(final_cross_sell_results[['country', 'antecedents','support', 'consequents', 'confidence', 'lift']])


print("\nFinal Up-Sell Candidates with Country Labels:")
print(final_up_sell_results[['country', 'antecedents', 'consequents', 'confidence', 'lift']])

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/mlxtend/frequent_patterns/fpcommon.py:161: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/mlxtend/frequent_patterns/fpcommon.py:161: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/mlxtend/frequent_patterns/fpcommon.py

Streaming output truncated to the last 5000 lines.
5219        0.45 15.06  
5220        0.45 15.06  
5221        0.45 15.06  
5222        0.78 15.04  
5223        0.78 15.04  
5224        0.78 15.04  
5225        0.78 15.04  
5226        0.58 15.04  
5227        0.58 15.04  
5228        0.58 15.04  
5229        0.58 15.04  
5230        0.90 14.91  
5231        0.90 14.91  
5232        0.64 14.91  
5233        0.64 14.91  
5234        0.83 14.87  
5235        0.83 14.87  
5236        0.83 14.87  
5237        0.83 14.87  
5238        0.83 14.87  
5239        0.83 14.87  
5240        0.83 14.87  
5241        0.83 14.87  
5242        0.83 14.87  
5243        0.83 14.87  
5244        0.83 14.87  
5245        0.83 14.87  
5246        0.83 14.87  
5247        0.83 14.87  
5248        0.77 14.87  
5249        0.77 14.87  
5250        0.77 14.87  
5251        0.77 14.87  
5252        0.77 14.87  
5253        0.38 14.87  
5254        0.38 14.87  
5255        0.38 14.87  
5256        0.38 14.87  

In [21]:
final_up_sell_results

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,country,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski,antecedent_category,consequent_category
0,France,PACK OF 6 SKULL PAPER CUPS,PACK OF 6 SKULL PAPER PLATES,0.06,0.05,0.04,0.71,14.29,1.00,0.04,3.32,0.99,0.62,0.70,0.77,Party Accessories,Party Accessories
1,France,LUNCH BAG APPLE DESIGN,LUNCH BOX WITH CUTLERY RETROSPOT,0.13,0.15,0.04,0.31,2.08,1.00,0.02,1.24,0.60,0.17,0.19,0.30,Lunch Bag,Lunch Bag
2,France,DOLLY GIRL LUNCH BOX,LUNCH BOX WITH CUTLERY RETROSPOT,0.10,0.15,0.03,0.31,2.04,1.00,0.02,1.22,0.57,0.14,0.18,0.25,Lunch Bag,Lunch Bag
3,France,LUNCH BAG RED RETROSPOT,LUNCH BOX WITH CUTLERY RETROSPOT,0.15,0.15,0.04,0.30,2.01,1.00,0.02,1.22,0.59,0.18,0.18,0.30,Lunch Bag,Lunch Bag
4,France,LUNCH BAG DOLLY GIRL DESIGN,DOLLY GIRL LUNCH BOX,0.09,0.10,0.03,0.29,2.90,1.00,0.02,1.27,0.72,0.16,0.21,0.27,Lunch Bag,Lunch Bag
5,France,LUNCH BAG WOODLAND,DOLLY GIRL LUNCH BOX,0.13,0.10,0.03,0.26,2.61,1.00,0.02,1.22,0.71,0.17,0.18,0.30,Lunch Bag,Lunch Bag
6,France,LUNCH BAG SPACEBOY DESIGN,DOLLY GIRL LUNCH BOX,0.12,0.10,0.03,0.26,2.56,1.00,0.02,1.21,0.69,0.16,0.17,0.28,Lunch Bag,Lunch Bag
7,France,LUNCH BAG SPACEBOY DESIGN,LUNCH BOX WITH CUTLERY RETROSPOT,0.12,0.15,0.03,0.23,1.55,1.00,0.01,1.11,0.40,0.11,0.10,0.21,Lunch Bag,Lunch Bag
8,France,LUNCH BAG WOODLAND,STRAWBERRY LUNCH BOX WITH CUTLERY,0.13,0.13,0.03,0.22,1.63,1.00,0.01,1.11,0.44,0.12,0.10,0.21,Lunch Bag,Lunch Bag
9,France,LUNCH BAG SPACEBOY DESIGN,STRAWBERRY LUNCH BOX WITH CUTLERY,0.12,0.13,0.03,0.21,1.57,1.00,0.01,1.10,0.41,0.11,0.09,0.20,Lunch Bag,Lunch Bag


In [17]:
db_config = {
    'host' : "projects-database.cxgcu68ksihx.us-east-1.rds.amazonaws.com",          # RDS endpoint, e.g., <your-db>.rds.amazonaws.com
    'port' : 5432,                       # Default port for PostgreSQL
    'dbname' : "postgres",               # Database name
    'user' : "postgres",                   # Username
    'password' : "DATA602_project"        # Password
}


try:
    # Connect to the PostgreSQL database
    connection = psycopg2.connect(**db_config)
    cursor = connection.cursor()

    # Create table if it doesn't exist
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS FPGrowth_cross_sell (
        country varchar(500),
        antecedents VARCHAR(4000),
        support NUMERIC,
        consequents VARCHAR(4000),
        confidence NUMERIC,
        lift NUMERIC
    );
    """)

    # Convert DataFrame to list of tuples
    data_to_insert = data_to_insert = [
        (row[0], row[1], float(row[2]), row[3], float(row[4]), float(row[5]))  # Explicitly convert to float
        for row in final_cross_sell_results[['country', 'antecedents', 'support', 'consequents', 'confidence', 'lift']].values.tolist()
    ]
    # Insert data into the table
    insert_query = "INSERT INTO FPGrowth_cross_sell (country, antecedents, support, consequents, confidence, lift) VALUES (%s, %s, %s, %s, %s, %s);"
    cursor.executemany(insert_query, data_to_insert)

    # Commit the transaction
    connection.commit()
    print("DataFrame data inserted successfully!")

except psycopg2.Error as e:
    print("Error:", e)
finally:
    # Close the cursor and connection
    if cursor:
        cursor.close()
    if connection:
        connection.close()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


DataFrame data inserted successfully!


In [22]:
db_config = {
    'host' : "projects-database.cxgcu68ksihx.us-east-1.rds.amazonaws.com",          # RDS endpoint, e.g., <your-db>.rds.amazonaws.com
    'port' : 5432,                       # Default port for PostgreSQL
    'dbname' : "postgres",               # Database name
    'user' : "postgres",                   # Username
    'password' : "DATA602_project"        # Password
}


try:
    # Connect to the PostgreSQL database
    connection = psycopg2.connect(**db_config)
    cursor = connection.cursor()

    # Create table if it doesn't exist
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS FPGrowth_up_sell (
        country varchar(500),
        antecedents VARCHAR(4000),
        support NUMERIC,
        consequents VARCHAR(4000),
        confidence NUMERIC,
        lift NUMERIC
    );
    """)

    # Convert DataFrame to list of tuples
    data_to_insert = data_to_insert = [
        (row[0], row[1], float(row[2]), row[3], float(row[4]), float(row[5]))  # Explicitly convert to float
        for row in final_up_sell_results[['country', 'antecedents', 'support', 'consequents', 'confidence', 'lift']].values.tolist()
    ]
    # Insert data into the table
    insert_query = "INSERT INTO FPGrowth_up_sell (country, antecedents, support, consequents, confidence, lift) VALUES (%s, %s, %s, %s, %s, %s);"
    cursor.executemany(insert_query, data_to_insert)

    # Commit the transaction
    connection.commit()
    print("DataFrame data inserted successfully!")

except psycopg2.Error as e:
    print("Error:", e)
finally:
    # Close the cursor and connection
    if cursor:
        cursor.close()
    if connection:
        connection.close()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


DataFrame data inserted successfully!
